## Introduction to Machine Learning, UZH FS18, Group Project

## Group 2: Barbara Capl, Mathias Lüthi, Pamela Matias, Stefanie Rentsch

In [1]:
import numpy as np
import pandas as pd
import matplotlib as pl
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
import sklearn as skl
from sklearn.svm import SVR
from sklearn.preprocessing import Imputer
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn import neural_network
from sklearn import neighbors
from functools import reduce
from sklearn.svm import SVR
from sklearn import neighbors
from functools import reduce
from sklearn.model_selection import train_test_split
import os
import datetime as dtfrom pandas.core import datetools


/home/andy/anaconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


# Import datasets


### Market Data from Wharton

In [2]:
# Import "big" wharton file -> wharton_ml
wharton = pd.read_csv('Data/wharton_ml3.csv', sep = ',')

# Rename  columns in wharton file
wharton.rename(columns = {'date': 'DATE_w', 'PERMNO': 'PERMNO_w'}, inplace = True)

# SPREAD seems to contain no data (nan), so I'm calculating spread via BID - ASK 
wharton['SPREAD'] = wharton['BID'] - wharton['ASK']

display(wharton.head())

,PERMNO_w,DATE_w,NAICS,DCLRDT,DLAMT,DLPDT,PAYDT,RCRDDT,SHRFLG,DISTCD,...,CFACSHR,ALTPRC,SPREAD,ALTPRCDT,RETX,vwretd,vwretx,ewretd,ewretx,sprtrn
0,10107,31/01/2006,511210.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,...,1.0,28.15,0.02,31/01/2006,0.076482,0.040044,0.038938,0.076354,0.075150,0.025467
1,10107,28/02/2006,511210.0,14/12/2005,NaN,NaN,09/03/2006,17/02/2006,0.0,1232.0,...,1.0,26.87,-0.01,28/02/2006,-0.045471,-0.001637,-0.003552,0.004836,0.003476,0.000453
2,10107,31/03/2006,511210.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,...,1.0,27.21,0.00,31/03/2006,0.012653,0.019053,0.017585,0.036978,0.035232,0.011065
3,10107,28/04/2006,511210.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,...,1.0,24.15,0.00,28/04/2006,-0.112459,0.012965,0.011494,0.009791,0.008497,0.012187
4,10107,31/05/2006,511210.0,27/03/2006,NaN,NaN,08/06/2006,17/05/2006,0.0,1232.0,...,1.0,22.65,0.00,31/05/2006,-0.062112,-0.031045,-0.033025,-0.044331,-0.045982,-0.030917


In [3]:
# REMOVE DATA THAT IS (PBLY) NOT NEEDED
wharton = wharton.drop(['RET', 'RETX', 'DLAMT', 'DLPDT', 'DCLRDT', 'DLRETX', 'DLRET', 'DLPRC', 'PAYDT', 'RCRDDT', 
                           'SHRFLG', 'DISTCD', 'DIVAMT', 'FACPR', 
                           'ACPERM', 'ACCOMP', 'SHRENDDT', 'FACSHR', 'ALTPRCDT'], axis = 1)

display(wharton.head())

,PERMNO_w,DATE_w,NAICS,BIDLO,ASKHI,PRC,VOL,BID,ASK,SHROUT,CFACPR,CFACSHR,ALTPRC,SPREAD,vwretd,vwretx,ewretd,ewretx,sprtrn
0,10107,31/01/2006,511210.0,26.28,28.15,28.15,13948013.0,28.15,28.13,10333369.0,1.0,1.0,28.15,0.02,0.040044,0.038938,0.076354,0.075150,0.025467
1,10107,28/02/2006,511210.0,26.39,28.04,26.87,11088149.0,26.87,26.88,10333369.0,1.0,1.0,26.87,-0.01,-0.001637,-0.003552,0.004836,0.003476,0.000453
2,10107,31/03/2006,511210.0,26.85,27.89,27.21,14514337.0,27.24,27.24,10225000.0,1.0,1.0,27.21,0.00,0.019053,0.017585,0.036978,0.035232,0.011065
3,10107,28/04/2006,511210.0,24.15,27.74,24.15,14689919.0,24.16,24.16,10201203.0,1.0,1.0,24.15,0.00,0.012965,0.011494,0.009791,0.008497,0.012187
4,10107,31/05/2006,511210.0,22.56,24.29,22.65,23651189.0,22.70,22.70,10201203.0,1.0,1.0,22.65,0.00,-0.031045,-0.033025,-0.044331,-0.045982,-0.030917


### Financial Ratios (from OLAT)

In [4]:
# Import financial ratios (downloaded from OLAT)
ratios_all = pd.read_csv('Data/Ratios.csv', sep = ',', converters = {'permno': str})

# Rename columns in ratios file
ratios_all.rename(columns = {'permno': 'PERMNO_r', 'public_date': 'DATE_r'}, inplace = True)

# REMOVE DATA THAT IS (PBLY) NOT NEEDED
ratios_all = ratios_all.drop(['adate', 'qdate'], axis = 1)

display(ratios_all.head())

,PERMNO_r,DATE_r,CAPEI,bm,evm,pe_op_basic,pe_op_dil,pe_exi,pe_inc,ps,...,sale_nwc,rd_sale,adv_sale,staff_sale,accrual,ptb,PEG_trailing,divyield,PEG_1yrforward,PEG_ltgforward
0,10107,31/01/2006,33.861,0.176,15.245,23.074,23.264,23.856,23.856,7.211,...,1.296,0.151,0.025,0.0,0.055,6.019,11.538,1.14%,15.506,2.109
1,10107,28/02/2006,31.756,0.163,14.951,21.496,21.846,22.392,22.392,6.713,...,1.323,0.151,0.025,0.0,0.036,6.281,10.280,1.34%,14.555,1.838
2,10107,31/03/2006,31.820,0.163,14.951,21.768,22.122,22.675,22.675,6.727,...,1.323,0.151,0.025,0.0,0.036,6.293,10.410,1.32%,14.739,1.842
3,10107,30/04/2006,28.176,0.163,14.951,19.320,19.634,20.125,20.125,5.957,...,1.323,0.151,0.025,0.0,0.036,5.573,9.239,1.49%,13.081,1.666
4,10107,31/05/2006,25.921,0.151,15.120,17.695,17.835,17.976,17.976,5.419,...,1.388,0.150,0.025,0.0,0.024,5.496,0.709,1.59%,-5.842,1.480


# Data preparation

### Convert Date and the PERMNO - Code, then rename columns

In [5]:
### DATE
# set date as datetime64
wharton['DATE_w'] = pd.to_datetime(wharton['DATE_w']).dt.date.astype('datetime64[ns]')
ratios_all['DATE_r'] = pd.to_datetime(ratios_all['DATE_r']).dt.date.astype('datetime64[ns]')

### PERMNO
# set PERMNO column as integer (int64) for ratios_all, because its type 'O'
ratios_all['PERMNO_r'] = ratios_all['PERMNO_r'].convert_objects(convert_numeric = True)

### RENAME COLUMNS AND SET PERMNO AND DATE AS INDEX
wharton.rename(columns = {'PERMNO_w': 'PERMNO', 'DATE_w': 'DATE'}, inplace = True)
ratios_all.rename(columns = {'PERMNO_r': 'PERMNO', 'DATE_r': 'DATE'}, inplace = True)


/home/andy/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use Series.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  


### Calculate Returns for each Company (PERMNO-group)

In [6]:
# make empty dataframe
grouped_by_permno = pd.DataFrame()

# Now follows a typical split-change-merge pattern to calculate the values for each PERMNO
# For each permno in df, do:
for df_key in wharton.groupby('PERMNO').groups:
    permno_group = wharton.groupby('PERMNO').get_group(df_key)
    permno_group['PRC_RET'] = np.log(permno_group['PRC']/ permno_group['PRC'].shift(1))
    grouped_by_permno = pd.concat([grouped_by_permno, permno_group])
wharton = grouped_by_permno

# Delete unused variables
del df_key, permno_group, grouped_by_permno

display(wharton['PRC_RET'].head())

/home/andy/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


0         NaN
1   -0.046537
2    0.012574
3   -0.119300
4   -0.064125
Name: PRC_RET, dtype: float64

### Convert calculated returns to movement

In [7]:
### Set 0, 1 for PRC_RET
def set_mov(mydata):
    if mydata['PRC_RET'] > 0:
        return 'UP'
    elif mydata['PRC_RET'] < 0:
        return 'DOWN'
    elif mydata['PRC_RET'] == 0:
        return 'UP'

wharton = wharton.assign(MOVEMENT = wharton.apply(set_mov, axis = 1))
wharton['PRC_MOV'] = wharton['MOVEMENT'].factorize()[0]
wharton = wharton.drop('MOVEMENT', axis =1)

display(wharton.PRC_MOV.head())

0   -1
1    0
2    1
3    0
4    0
Name: PRC_MOV, dtype: int64

### Set all dates to first day of month

In [8]:
# Because wharton and ratios dataset have sometimes differing "end" dates of month
wharton['DATE'] = wharton['DATE'].apply(lambda dt: dt.replace(day=1))
ratios_all['DATE'] = ratios_all['DATE'].apply(lambda dt: dt.replace(day=1))


## Join the two prepared datasets (outer join)

In [9]:
# Set index
wharton = wharton.set_index(['PERMNO', 'DATE'])
ratios_all = ratios_all.set_index(['PERMNO', 'DATE'])

# join datasets
joined_dataset = wharton.join(ratios_all, how = 'outer')

# reset index
joined_dataset = joined_dataset.reset_index()

display(joined_dataset.head())



,PERMNO,DATE,NAICS,BIDLO,ASKHI,PRC,VOL,BID,ASK,SHROUT,...,sale_nwc,rd_sale,adv_sale,staff_sale,accrual,ptb,PEG_trailing,divyield,PEG_1yrforward,PEG_ltgforward
0,10107,2006-01-01,511210.0,26.28,28.15,28.15,13948013.0,28.15,28.13,10333369.0,...,1.296,0.151,0.025,0.0,0.055,6.019,11.538,1.14%,15.506,2.109
1,10107,2006-02-01,511210.0,26.39,28.04,26.87,11088149.0,26.87,26.88,10333369.0,...,1.323,0.151,0.025,0.0,0.036,6.281,10.280,1.34%,14.555,1.838
2,10107,2006-03-01,511210.0,26.85,27.89,27.21,14514337.0,27.24,27.24,10225000.0,...,1.323,0.151,0.025,0.0,0.036,6.293,10.410,1.32%,14.739,1.842
3,10107,2006-04-01,511210.0,24.15,27.74,24.15,14689919.0,24.16,24.16,10201203.0,...,1.323,0.151,0.025,0.0,0.036,5.573,9.239,1.49%,13.081,1.666
4,10107,2006-05-01,511210.0,22.56,24.29,22.65,23651189.0,22.70,22.70,10201203.0,...,1.388,0.150,0.025,0.0,0.024,5.496,0.709,1.59%,-5.842,1.480


### Extract all rows of each PERMNO-Group except for first one (because it is Nan)

In [10]:
# Prepare empty dataframe
grouped_by_permno = pd.DataFrame()

# Group by PERMNO-Code and then remove first (0) row because it is nan in every PRC_RET group
for df_key in joined_dataset.groupby('PERMNO').groups:
    permno_group = joined_dataset.groupby('PERMNO').get_group(df_key)
    permno_group = permno_group[1:]
    grouped_by_permno = pd.concat([grouped_by_permno, permno_group])
joined_dataset = grouped_by_permno

# Delete unused variables
del df_key, permno_group, grouped_by_permno

display(joined_dataset['PRC_RET'].head())

1   -0.046537
2    0.012574
3   -0.119300
4   -0.064125
5    0.028294
Name: PRC_RET, dtype: float64

### Prepare column "divyield"

In [11]:
display(joined_dataset['divyield'].head())

# Remove percentages in row "divyield" and divide with 100 (so its decimal percentage) with string split
joined_dataset['divyield'] = joined_dataset['divyield'].str.rstrip('%').astype('float')/100

display(joined_dataset['divyield'].head())


1    1.34%
2    1.32%
3    1.49%
4    1.59%
5    1.55%
Name: divyield, dtype: object

1    0.0134
2    0.0132
3    0.0149
4    0.0159
5    0.0155
Name: divyield, dtype: float64

### Prepare Response Column with lagged Returns

In [12]:
# New Column with lagged movement of prices, because we want to have lagged prices as response vector later
# because we want to forecast prices of tomorrow, not of today with our model
joined_dataset['NEXT_DAY_PREDICTION'] = joined_dataset['PRC_MOV']
grouped_by_permno = pd.DataFrame()

for df_key in joined_dataset.groupby('PERMNO').groups:
    permno_group = joined_dataset.groupby('PERMNO').get_group(df_key)
    permno_group['NEXT_DAY_PREDICTION'] = permno_group['NEXT_DAY_PREDICTION'].shift(-1)
    permno_group = permno_group[pd.notnull(permno_group['NEXT_DAY_PREDICTION'])]
    grouped_by_permno = pd.concat([grouped_by_permno, permno_group])
joined_dataset = grouped_by_permno

# Delete unused variables
del df_key, permno_group, grouped_by_permno

display(joined_dataset['NEXT_DAY_PREDICTION'].head())

/home/andy/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


1    1.0
2    0.0
3    0.0
4    1.0
5    1.0
Name: NEXT_DAY_PREDICTION, dtype: float64

### Change Datatype of "Date"

In [13]:
# Make date numeric because RF can not use otherwise
joined_dataset['DATE'] = joined_dataset['DATE'].astype('int64')


# Random Forest

#### -----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
### 1. Run Forest without removing Features below
### 2. Remove Features that are "useless"
### 3. Run the Forest again and see the score changing depending on what columns are removed
#### -----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------



####   


### Remove Variables found "unnecessary" in the Random Forest Analysis

In [14]:
# Remove a lot of variables that were foud useless in the RF process
joined_dataset = joined_dataset.drop(['debt_at', 'cash_conversion', 'at_turn', 'adv_sale', 'rect_turn',
       'de_ratio', 'divyield', 'pretret_earnat', 'gpm', 'sale_nwc',
       'pretret_noa', 'npm', 'intcov_ratio', 'efftax', 'roe', 'roa', 'PRC_MOV',
       'sale_invcap', 'debt_ebitda', 'short_debt', 'debt_capital', 'NAICS',
       'ocf_lct', 'int_totdebt', 'PERMNO', 'totdebt_invcap', 'equity_invcap',
       'lt_debt', 'staff_sale', 'cash_debt', 'CFACPR', 'dltt_be','BID', 'pcf', 'SHROUT', 'ALTPRC', 'GProf', 'pe_op_dil', 'cfm',
       'accrual', 'bm', 'lt_ppent', 'evm', 'ASK', 'curr_debt', 'int_debt',
       'quick_ratio', 'inv_turn', 'roce', 'ASKHI', 'opmbd', 'ptpm',
       'sale_equity', 'fcf_ocf', 'capital_ratio', 'aftret_eq', 'cash_lt',
       'cash_ratio', 'pay_turn', 'curr_ratio', 'rect_act', 'opmad',
       'debt_invcap', 'profit_lct', 'invt_act', 'intcov', 'aftret_invcapx',
       'debt_assets', 'aftret_equity', 'SPREAD', 'rd_sale', 'CFACSHR'], axis = 1)

display(joined_dataset.size)

73899

### Fill missing Values

In [15]:
### Fill missing value with sklearn IMPUTER (fills with mean)
imp = Imputer(missing_values=np.nan, strategy = 'mean' , axis=0)
imputed_dataset = pd.DataFrame(imp.fit_transform(joined_dataset))
imputed_dataset.columns = joined_dataset.columns
imputed_dataset.index = joined_dataset.index


### Feature Matrix and Response Vector 

In [16]:
# Extract labels of features
labels_of_features = imputed_dataset.columns[:-1]

# X1 is the feature matrix
X1 = imputed_dataset.iloc[:, :-1]
display(X1.head())


,DATE,BIDLO,PRC,VOL,vwretd,vwretx,ewretd,ewretx,sprtrn,PRC_RET,CAPEI,pe_op_basic,pe_exi,pe_inc,ps,dpr,ptb,PEG_trailing,PEG_1yrforward,PEG_ltgforward
1,1.138752e+18,26.39,26.87,11088149.0,-0.001637,-0.003552,0.004836,0.003476,0.000453,-0.046537,31.756,21.496,22.392,22.392,6.713,0.264,6.281,10.280,14.555,1.838
2,1.141171e+18,26.85,27.21,14514337.0,0.019053,0.017585,0.036978,0.035232,0.011065,0.012574,31.820,21.768,22.675,22.675,6.727,0.264,6.293,10.410,14.739,1.842
3,1.143850e+18,24.15,24.15,14689919.0,0.012965,0.011494,0.009791,0.008497,0.012187,-0.119300,28.176,19.320,20.125,20.125,5.957,0.264,5.573,9.239,13.081,1.666
4,1.146442e+18,22.56,22.65,23651189.0,-0.031045,-0.033025,-0.044331,-0.045982,-0.030917,-0.064125,25.921,17.695,17.976,17.976,5.419,0.259,5.496,0.709,-5.842,1.480
5,1.149120e+18,21.51,23.30,19980809.0,-0.000386,-0.001881,-0.008479,-0.010474,0.000087,0.028294,26.301,18.203,18.492,18.492,5.498,0.259,5.577,0.730,-6.010,1.522


In [17]:
# y1 is the response vector
y1 = imputed_dataset.iloc[:, -1]
display(y1.head())

1    1.0
2    0.0
3    0.0
4    1.0
5    1.0
Name: NEXT_DAY_PREDICTION, dtype: float64

### Train - / Test - Split

In [18]:
# Do the train - test- split
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size = 0.2, random_state = 0, stratify = y1)


In [19]:
# Check if there is the approximately same percentage of '1' i both training and test response vector
display(y1_train.sum() / y1_train.size)
display(y1_test.sum() / y1_test.size)


0.5602131438721136

0.5610795454545454

### Standardize Variables

In [21]:
# Standardization with sklearn StandardScaler
standard_scaler = preprocessing.StandardScaler().fit(X1_train)
X1_train = standard_scaler.transform(X1_train)
X1_test = standard_scaler.transform(X1_test)


### Feature Selection with Random Forest

In [23]:
my_rainy_forest = RandomForestClassifier(random_state = 1)
my_rainy_forest.max_depth = 8
my_rainy_forest.fit(X1_train, y1_train)

# Check features for their importance for the prediction
features_importances = my_rainy_forest.feature_importances_



In [25]:
# sort features in line with their importance for the prediction
indices = np.argsort(features_importances)[::-1]



In [26]:
# print best 15 features and delete features above and start the RF again
i = 0
n = 15
for i in range(n):
    print('{0:2d} {1:7s} {2:6.4f}'.format(i+1, labels_of_features[indices[i]], features_importances[indices[i]]))
del i,n

 1 vwretx  0.0973
 2 ewretd  0.0914
 3 sprtrn  0.0905
 4 vwretd  0.0881
 5 DATE    0.0786
 6 ewretx  0.0698
 7 PEG_ltgforward 0.0478
 8 BIDLO   0.0433
 9 PRC_RET 0.0404
10 ptb     0.0380
11 PEG_1yrforward 0.0370
12 CAPEI   0.0370
13 PRC     0.0366
14 VOL     0.0348
15 ps      0.0316


### Prediction of Response Vector with Random Forest

In [34]:
# Test prediction of y1 with the test feature matrix: gives the prediction vector
prediction1 = my_rainy_forest.predict(X1_test)

display(prediction1[1:5])



array([1., 0., 1., 1.])

In [35]:
# For comparison:

display(y1_test[1:5])

2890    0.0
478     1.0
2816    0.0
1331    1.0
Name: NEXT_DAY_PREDICTION, dtype: float64

### Assess Results of Prediction

In [44]:
# Calculate percentage of of ones in the test response vector
y_test_score = print('Ratioof Ones (Test)    =  ' + str(y1_test.sum() / y1_test.size))
# Just to be sure the ones are distributed ca.the same in test and train response vector, check this:
y_train_score = print('Ratio of Ones (Train)  =  ' + str(y1_train.sum() / y1_train.size))
# Calculate precentage of ones predicted with the model
prediction_score = print('Score (Prediction)     =  ' + str(prediction1.sum() / prediction1.size))


Ratioof Ones (Test)    =  0.5610795454545454
Ratio of Ones (Train)  =  0.5602131438721136
Score (Prediction)     =  0.6860795454545454


In [45]:
# Calculate the score surplus above the test-set response vector score
(prediction1.sum()/prediction1.size) - (y1_test.sum()/y1_test.size)


0.125

### Cross Validation: tbc